# Refua Antibody Design Notebook: SARS-CoV-2 RBD

Updated antibody workflow using vanilla Refua objects and automatic notebook rendering.

## Workflow

1. Define antigen and epitope intent.
2. Build heavy/light binders with `BinderDesigns.antibody(...)`.
3. Compose a `Complex` and run fold/design.


In [1]:
%load_ext refua_notebook
import refua_notebook

refua_notebook.activate()


Refua notebook extension loaded. Refua objects will now display as interactive widgets.


True

In [2]:
from refua import BinderDesigns, Complex, Protein


[15:15:09] Initializing Normalizer


## Define antigen and CDR settings


In [3]:
RBD_SEQUENCE = (
    "RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF"
)

EPITOPE = "118..190"
HEAVY_CDR_LENGTHS = (12, 10, 14)
LIGHT_CDR_LENGTHS = (10, 9, 9)


## Build antibody design spec


In [4]:
antigen = Protein(
    RBD_SEQUENCE,
    ids="A",
    binding_types={"binding": EPITOPE},
)

binder_pair = BinderDesigns.antibody(
    heavy_cdr_lengths=HEAVY_CDR_LENGTHS,
    light_cdr_lengths=LIGHT_CDR_LENGTHS,
    heavy_id="H",
    light_id="L",
)

design_spec = Complex([antigen, *binder_pair], name="rbd_fab_epitope_design")


In [5]:
{
    "heavy_spec": binder_pair.heavy.sequence,
    "light_spec": binder_pair.light.sequence,
}


{'heavy_spec': 'QVQLVESGGGLVQPGGSLRLSCAAS12WYRQAPGKEREWV10ISSGGSTYYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYC14WGQGTLVTVSS',
 'light_spec': 'DIQMTQSPSSLSASVGDRVTITCRAS10WYQQKPGKAPKLLIY9ASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYC9FGGGTKVEIK'}

## Design narrative: from epitope intent to actionable candidates

This setup mirrors an early-stage antibody campaign where we first encode **intent** before spending compute on full structural runs.

What we are doing conceptually:

- We mark an antigen region (`118..190`) as the preferred interaction hotspot.
- We define heavy/light CDR window sizes to control where sequence diversity is allowed.
- We ask Refua to generate design-ready representations (`binder_specs` and feature tensors) that can be screened, compared, and iterated.

How to read the next outputs:

- `result.binder_specs`: the heavy/light design templates, not final experimentally validated sequences.
- `result.chain_design_summary()`: per-chain coverage of designable vs. fixed regions; useful for sanity checks across design variants.
- `result.features`: model-ready tensors/keys you can persist for reproducibility and downstream ranking workflows.

Practical strategy: run this lightweight pass for many CDR settings first, shortlist promising designs by consistency and mask coverage, then run full Boltz2 structure folds only on the top subset.


## Run a design pass


In [6]:
result = design_spec.fold()

result


/Users/jj/code/refua/src/refua/boltz/api.py:1509: RuntimeWarning: cuEquivariance kernels requested but cuequivariance_torch is not installed. Falling back to use_kernels=False.
  self._disable_kernels(


FoldResult(backend='boltz+boltzgen', structure=StructurePrediction(structure=StructureV2(atoms=array([('N', [ 49.298275 ,  -7.741211 ,  44.43242  ],  True, 0., 1.),
       ('CA', [ 48.19831  ,  -7.4303226,  43.519226 ],  True, 0., 1.),
       ('C', [ 48.767582 ,  -6.918825 ,  42.208633 ],  True, 0., 1.), ...,
       ('CD', [-18.678879 ,  34.27173  , -18.397236 ],  True, 0., 1.),
       ('CE', [-19.260725 ,  35.46646  , -17.682423 ],  True, 0., 1.),
       ('NZ', [-18.209455 ,  36.235603 , -16.957104 ],  True, 0., 1.)],
      dtype=[('name', '<U4'), ('coords', '<f4', (3,)), ('is_present', '?'), ('bfactor', '<f4'), ('plddt', '<f4')]), bonds=array([],
      dtype=[('chain_1', '<i4'), ('chain_2', '<i4'), ('res_1', '<i4'), ('res_2', '<i4'), ('atom_1', '<i4'), ('atom_2', '<i4'), ('type', 'i1')]), residues=array([('ARG',  3,   0,    0, 11,    1,    4,  True,  True),
       ('VAL', 21,   1,   11,  7,   12,   15,  True,  True),
       ('GLN',  7,   2,   18,  9,   19,   22,  True,  True),
       ('PRO', 16,   3,   27,  7,   28,   31,  True,  True),
       ('THR', 18,   4,   34,  7,   35,   38,  True,  True),
       ('GLU',  8,   5,   41,  9,   42,   45,  True,  True),
       ('SER', 17,   6,   50,  6,   51,   54,  True,  True),
       ('ILE', 11,   7,   56,  8,   57,   60,  True,  True),
       ('VAL', 21,   8,   64,  7,   65,   68,  True,  True),
       ('ARG',  3,   9,   71, 11,   72,   75,  True,  True),
       ('PHE', 15,  10,   82, 11,   83,   86,  True,  True),
       ('PRO', 16,  11,   93,  7,   94,   97,  True,  True),
       ('ASN',  4,  12,  100,  8,  101,  104,  True,  True),
       ('ILE', 11,  13,  108,  8,  109,  112,  True,  True),
       ('THR', 18,  14,  116,  7,  117,  120,  True,  True),
       ('ASN',  4,  15,  123,  8,  124,  127,  True,  True),
       ('LEU', 12,  16,  131,  8,  132,  135,  True,  True),
       ('CYS',  6,  17,  139,  6,  140,  143,  True,  True),
       ('PRO', 16,  18,  145,  7,  146,  149,  True,  True),
       ('PHE', 15,  19,  152, 11,  153,  156,  True,  True),
       ('GLY',  9,  20,  163,  4,  164,  164,  True,  True),
       ('GLU',  8,  21,  167,  9,  168,  171,  True,  True),
       ('VAL', 21,  22,  176,  7,  177,  180,  True,  True),
       ('PHE', 15,  23,  183, 11,  184,  187,  True,  True),
       ('ASN',  4,  24,  194,  8,  195,  198,  True,  True),
       ('ALA',  2,  25,  202,  5,  203,  206,  True,  True),
       ('THR', 18,  26,  207,  7,  208,  211,  True,  True),
       ('ARG',  3,  27,  214, 11,  215,  218,  True,  True),
       ('PHE', 15,  28,  225, 11,  226,  229,  True,  True),
       ('ALA',  2,  29,  236,  5,  237,  240,  True,  True),
       ('SER', 17,  30,  241,  6,  242,  245,  True,  True),
       ('VAL', 21,  31,  247,  7,  248,  251,  True,  True),
       ('TYR', 20,  32,  254, 12,  255,  258,  True,  True),
       ('ALA',  2,  33,  266,  5,  267,  270,  True,  True),
       ('TRP', 19,  34,  271, 14,  272,  275,  True,  True),
       ('ASN',  4,  35,  285,  8,  286,  289,  True,  True),
       ('ARG',  3,  36,  293, 11,  294,  297,  True,  True),
       ('LYS', 13,  37,  304,  9,  305,  308,  True,  True),
       ('ARG',  3,  38,  313, 11,  314,  317,  True,  True),
       ('ILE', 11,  39,  324,  8,  325,  328,  True,  True),
       ('SER', 17,  40,  332,  6,  333,  336,  True,  True),
       ('ASN',  4,  41,  338,  8,  339,  342,  True,  True),
       ('CYS',  6,  42,  346,  6,  347,  350,  True,  True),
       ('VAL', 21,  43,  352,  7,  353,  356,  True,  True),
       ('ALA',  2,  44,  359,  5,  360,  363,  True,  True),
       ('ASP',  5,  45,  364,  8,  365,  368,  True,  True),
       ('TYR', 20,  46,  372, 12,  373,  376,  True,  True),
       ('SER', 17,  47,  384,  6,  385,  388,  True,  True),
       ('VAL', 21,  48,  390,  7,  391,  394,  True,  True),
       ('LEU', 12,  49,  397,  8,  398,  401,  True,  True),
       ('TYR', 20,  50,  405, 12,  406,  409,  True,  True),
       ('ASN',  4,  51,  417,  8,  418,  421,  True,  True),
       ('SER', 17

In [7]:
result.binder_specs


{'H': 'QVQLVESGGGLVQPGGSLRLSCAAS12WYRQAPGKEREWV10ISSGGSTYYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYC14WGQGTLVTVSS',
 'L': 'DIQMTQSPSSLSASVGDRVTITCRAS10WYQQKPGKAPKLLIY9ASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYC9FGGGTKVEIK'}

In [8]:
result.chain_design_summary()


({'chain_id': 'A',
  'molecule_type': 'protein',
  'residue_count': 223,
  'design_residue_count': 0,
  'binding_residue_count': 73,
  'not_binding_residue_count': 0,
  'sequence': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNF'},
 {'chain_id': 'H',
  'molecule_type': 'protein',
  'residue_count': 130,
  'design_residue_count': 36,
  'binding_residue_count': 0,
  'not_binding_residue_count': 0,
  'sequence': 'QVQLVESGGGLVQPGGSLRLSCAASGGGGGGGGGGGGWYRQAPGKEREWVGGGGGGGGGGISSGGSTYYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCGGGGGGGGGGGGGGWGQGTLVTVSS'},
 {'chain_id': 'L',
  'molecule_type': 'protein',
  'residue_count': 117,
  'design_residue_count': 28,
  'binding_residue_count': 0,
  'not_binding_residue_count': 0,
  'sequence': 'DIQMTQSPSSLSASVGDRVTITCRASGGGGGGGGGGWYQQKPGKAPKLLIYGGGGGGGGGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDF